### **MonthlyMean rainfall in Aosta Valley/Italy**

In [ ]:
#Import earth engine
import ee

#Trigger the authentication flow
ee.Authenticate()

#Initialize the library
ee.Initialize(project='modis-410817')

In [ ]:
# Import geemap
import geemap as emap

In [ ]:
# Create a region of interest (ROI)
roi = ee.Geometry.Rectangle([7.224,45.479, 7.448,45.564])

# Define the image collection
era5land = ee.ImageCollection("ECMWF/ERA5/DAILY") \
    .select('total_precipitation') \
    .filter(ee.Filter.calendarRange(2000, 2020, 'year')) \
    .filter(ee.Filter.Or(
        ee.Filter.calendarRange(1, 3, 'month'),
        ee.Filter.calendarRange(10, 12, 'month')
    )) \
    .filterBounds(roi) \
    .map(lambda img: img.set({'month': ee.Date(ee.Number(img.get('system:time_start'))).get('month'),
                               'year': ee.Date(ee.Number(img.get('system:time_start'))).get('year')}))


# Multiply precipitation values by 1000
era5P = era5land.map(lambda image: image.multiply(1000)
                                         .set('year', image.get('year'))
                                         .set('month', image.get('month'))
                                         .set('system:time_start', image.get('system:time_start')))

In [ ]:
# Define the list of months and years
#months = [1, 2, 3, 10, 11, 12]   ## To get data from 2001 to 2020
months = [ 10, 11, 12]   ## To get data in 2000
#months = [ 1, 2, 3]   ## To get data in 2020

#years = list(range(2001, 2020))   ## To get data from 2001 to 2020
years = list(range(2000, 2001))   ## To get data in 2000
#years = list(range(2020, 2021))   ## To get data in 2020

byYearMonth = ee.ImageCollection.fromImages(
    [era5P.filter(ee.Filter.calendarRange(y, y, 'year'))
             .filter(ee.Filter.calendarRange(m, m, 'month'))
             #.select('total_precipitation')
             .mean()
             .set('year', y)
             .set('month', m)
             .set('system:time_start', ee.Date.fromYMD(y, m, 1))
     for y in years for m in months]
)

In [ ]:
# Get the image IDs
image_ids = byYearMonth.aggregate_array('system:index').getInfo()
print('Total images', len(image_ids))

Total images 24


In [ ]:
# Get a list of images
image_list = byYearMonth.toList(byYearMonth.size())

#Clip for displaying in map
img = ee.Image(image_list.get(0)).clip(roi)

In [ ]:
# Visualization palette for total precipitation
visTp = {
  'min': 0.0,
  'max': 500,
  'palette': ['ffffff', '00ffff', '0080ff', 'da00ff', 'ffa400', 'ff0000']
}

In [ ]:
#Create a center the map object
Map = emap.Map()
Map.center_object(roi, zoom=12)
# Add the layers to the map object.
Map.addLayer(img,visTp,'Rainfall')

# Draw the rectangle on the map
Map.addLayer(roi, {'color': 'red'}, 'roi')

#Add a layer control panel to the map
Map.addLayerControl()

#Display the map
Map

In [ ]:
# Export loop
exportArgs = {
    'folder': "RainfallMean_Aosta Valle",
    'region': roi,
    'scale': 500,
    'fileFormat': "GeoTIFF",
    'crs': 'EPSG:4326',
    'maxPixels': 1e10,
    'formatOptions': {
        'cloudOptimized': True
    }
}

# Convert the image collection into a list
imgList = byYearMonth.toList(byYearMonth.size())
numImg = imgList.size().getInfo()

for i in range(numImg):
    # Get the images from the list
    image = ee.Image(imgList.get(i))

    # Extract month and year information from image properties
    month = image.get('month')
    year = image.get('year')

    # Get the date of the image
    #date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

    exportArgs['image'] = image.select('total_precipitation')
    exportArgs['description'] = 'RainfallMean_Aosta Valle_{}_{}_{}'.format(year.getInfo(), month.getInfo(), i + 1)
    exportArgs['fileNamePrefix'] = '{}_{}'.format(year.getInfo(), month.getInfo())

    # Export the images
    task = ee.batch.Export.image.toDrive(**exportArgs)
    task.start()
    print('Started task: ', i + 1)

Started task:  1
Started task:  2
Started task:  3
Started task:  4
Started task:  5
Started task:  6
Started task:  7
Started task:  8
Started task:  9
Started task:  10
Started task:  11
Started task:  12
